In [12]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import datetime
import matplotlib.pyplot as plt
import contextily as cx

## 读取数据 #df

In [13]:
path = 'process data/dataset2/'
df = pd.read_csv(path+'dataset_2.csv', usecols=[0,3,4])
df

,device_id,device_home_latitude,device_home_longitude
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
1,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
3,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
4,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
...,...,...,...
59839133,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839134,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839135,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839136,b4797889-7ba8-6302-2099-7fd7d0cf6e40,22.266985,114.234298


In [14]:
df = df.drop_duplicates()
df

,device_id,device_home_latitude,device_home_longitude
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,22.407392,114.224594
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,22.324517,114.160602
15,e9872505-727b-42f7-b4d5-efe02dc948d7,22.336465,114.245417
...,...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,22.367530,114.127846
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,22.469127,114.006127
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,22.257824,114.165734
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,22.290000,114.150000


In [ ]:
duplicates = df.duplicated(subset='device_id')
df[duplicates]
## 无重复deviceid

,device_id,device_home_latitude,device_home_longitude


In [ ]:
device_cnt = np.size(df['device_id'].unique())
device_cnt
## 有住址经纬度的用户有 105037 个

105037

In [ ]:
# ## 没有重复值，保存数据：
# df.to_csv(path + "devics_lat_lon.csv", index=False)

## 将 device经纬度 转换为 GeoDataFrame #device_lat_lon_gdf

In [15]:
geometry = gpd.points_from_xy(df['device_home_longitude'], df['device_home_latitude'])
device_lat_lon_gdf = gpd.GeoDataFrame(df, geometry=geometry)
device_lat_lon_gdf.crs = 'EPSG:4326'
device_lat_lon_gdf

,device_id,device_home_latitude,device_home_longitude,geometry
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036,POINT (114.25804 22.37802)
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828,POINT (114.13683 22.36588)
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,22.407392,114.224594,POINT (114.22459 22.40739)
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,22.324517,114.160602,POINT (114.16060 22.32452)
15,e9872505-727b-42f7-b4d5-efe02dc948d7,22.336465,114.245417,POINT (114.24542 22.33646)
...,...,...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,22.367530,114.127846,POINT (114.12785 22.36753)
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,22.469127,114.006127,POINT (114.00613 22.46913)
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,22.257824,114.165734,POINT (114.16573 22.25782)
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,22.290000,114.150000,POINT (114.15000 22.29000)


In [16]:
device_lat_lon_gdf = device_lat_lon_gdf.drop(['device_home_latitude','device_home_longitude'], axis=1)
device_lat_lon_gdf

,device_id,geometry
0,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
2,f6580b80-1451-48b9-b99f-685c17a54fc8,POINT (114.13683 22.36588)
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,POINT (114.22459 22.40739)
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,POINT (114.16060 22.32452)
15,e9872505-727b-42f7-b4d5-efe02dc948d7,POINT (114.24542 22.33646)
...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,POINT (114.12785 22.36753)
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,POINT (114.00613 22.46913)
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,POINT (114.16573 22.25782)
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,POINT (114.15000 22.29000)


## 读取building shape file #building_shp

In [17]:
## 读取香港building shape file
shapefile = gpd.read_file('Building_SHP/BUILDING_STRUCTURE.shp', encoding="GBK")
shapefile = shapefile.to_crs('EPSG:4326')
shapefile

,OBJECTID,BUILDINGST,BUILDINGCS,BUILDING00,CATEGORY,STATUS,STATUSDATE,OFFICIALBU,OFFICIAL00,NUMABOVEGR,NUMBASEMEN,TOPHEIGHT,BASEHEIGHT,GROSSFLOOR,RECORDCREA,RECORDUPDA,SHAPE_Leng,SHAPE_Area,geometry
0,1,243561,0162608928T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,17.431311,14.453672,"POLYGON ((113.84091 22.21870, 113.84090 22.218..."
1,2,243562,0163008922T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,12.960214,9.240269,"POLYGON ((113.84094 22.21863, 113.84095 22.218..."
2,3,243563,0163108931T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,22.352643,31.015898,"POLYGON ((113.84090 22.21867, 113.84091 22.218..."
3,4,243564,0163608930T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,18.595424,21.319706,"POLYGON ((113.84097 22.21871, 113.84101 22.218..."
4,5,243565,0163808933T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,19.609693,20.145564,"POLYGON ((113.84102 22.21870, 113.84101 22.218..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340747,340748,403798,6296144493T20190806,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20190806,20190829,25.017263,39.012734,"POLYGON ((114.43641 22.53999, 114.43638 22.539..."
340748,340749,4963,6296144621T20050430,T,9,A,None,None,None,NaN,NaN,10.1,None,NaN,20050430,20050725,34.926430,66.714851,"POLYGON ((114.43640 22.54116, 114.43640 22.541..."
340749,340750,4952,6299144635T20050430,T,9,A,None,None,澶╁悗瀹,NaN,NaN,9.7,None,NaN,20050430,20050725,43.341399,117.309442,"POLYGON ((114.43668 22.54119, 114.43659 22.541..."
340750,340751,5040,6299744554T20050430,T,9,A,None,None,None,NaN,NaN,17.5,None,NaN,20050430,20050725,22.022560,29.099489,"POLYGON ((114.43675 22.54050, 114.43670 22.540..."


In [18]:
building_shp = shapefile[shapefile['BUILDING00'] == 'T']
building_shp = building_shp[['BUILDINGST', 'geometry']]
building_shp

,BUILDINGST,geometry
0,243561,"POLYGON ((113.84091 22.21870, 113.84090 22.218..."
1,243562,"POLYGON ((113.84094 22.21863, 113.84095 22.218..."
2,243563,"POLYGON ((113.84090 22.21867, 113.84091 22.218..."
3,243564,"POLYGON ((113.84097 22.21871, 113.84101 22.218..."
4,243565,"POLYGON ((113.84102 22.21870, 113.84101 22.218..."
...,...,...
340747,403798,"POLYGON ((114.43641 22.53999, 114.43638 22.539..."
340748,4963,"POLYGON ((114.43640 22.54116, 114.43640 22.541..."
340749,4952,"POLYGON ((114.43668 22.54119, 114.43659 22.541..."
340750,5040,"POLYGON ((114.43675 22.54050, 114.43670 22.540..."


## 关联building_shp和device_lat_lon_gdf #merged

In [19]:
merged = gpd.sjoin(building_shp, device_lat_lon_gdf, how='inner', predicate='intersects')
# merged = merged.drop('index_right', axis=1)
merged

,BUILDINGST,geometry,index_right,device_id
310,246850,"POLYGON ((113.85992 22.25518, 113.85987 22.255...",11403822,9d84fd00-873a-4473-9c17-42bbf0402455
483,186396,"POLYGON ((113.86199 22.25448, 113.86187 22.254...",1922445,7da1b9a7-3908-475e-a766-638b5d760eb7
573,246925,"POLYGON ((113.86239 22.25465, 113.86238 22.254...",31462,4b625d9c-fc79-465f-8315-8f39cca15e0e
698,174391,"POLYGON ((113.86287 22.24768, 113.86284 22.247...",1748385,8f47afdb-70aa-438b-a65b-c23e70f1a7be
805,196917,"POLYGON ((113.86318 22.25502, 113.86318 22.255...",798841,eb9023f1-b06a-4e4f-8a53-cae203f32b24
...,...,...,...,...
338948,60539,"POLYGON ((114.31958 22.44845, 114.31961 22.448...",244952,0efd6c0d-6f95-47b9-a4b6-f0724a970a8f
339357,343166,"POLYGON ((114.32680 22.43128, 114.32686 22.431...",5079117,908023fa-e3a1-4a58-8b4c-c8b358ce252a
339447,45990,"POLYGON ((114.32859 22.45994, 114.32859 22.460...",4066645,9c63161a-3895-4dc9-820d-2de8fbeae333
339848,138660,"POLYGON ((114.35282 22.34764, 114.35281 22.347...",4499660,b2a074a2-fd10-463b-adc3-14288ae32f9c


In [ ]:
merged[merged['index_right'].duplicated()]

,BUILDINGST,geometry,index_right,device_id


In [20]:
unmapped = gpd.sjoin(building_shp, device_lat_lon_gdf, how='right', predicate='intersects')
unmapped

,index_left,BUILDINGST,device_id,geometry
0,NaN,NaN,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
2,NaN,NaN,f6580b80-1451-48b9-b99f-685c17a54fc8,POINT (114.13683 22.36588)
10,301932.0,98228.0,f68a046d-48d6-41bc-a8dc-8210b026d2a7,POINT (114.22459 22.40739)
14,NaN,NaN,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,POINT (114.16060 22.32452)
15,NaN,NaN,e9872505-727b-42f7-b4d5-efe02dc948d7,POINT (114.24542 22.33646)
...,...,...,...,...
54706209,NaN,NaN,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,POINT (114.12785 22.36753)
55123313,NaN,NaN,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,POINT (114.00613 22.46913)
55997466,NaN,NaN,a4158a52-3c61-4f02-abbb-f035faa6faee,POINT (114.16573 22.25782)
56447442,NaN,NaN,eebeebff-812f-4be2-aaa4-94f86c528427,POINT (114.15000 22.29000)


In [21]:
## 105037 个有住址的用户中， 可以获得building 的有 19435 个, 4/5 用户无法映射到 building
unmapped = unmapped[unmapped['index_left'].isna()]
unmapped

,index_left,BUILDINGST,device_id,geometry
0,NaN,NaN,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
2,NaN,NaN,f6580b80-1451-48b9-b99f-685c17a54fc8,POINT (114.13683 22.36588)
14,NaN,NaN,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,POINT (114.16060 22.32452)
15,NaN,NaN,e9872505-727b-42f7-b4d5-efe02dc948d7,POINT (114.24542 22.33646)
18,NaN,NaN,e99a8a9e-712e-4e15-9e59-cecfe7142aa4,POINT (114.15255 22.27501)
...,...,...,...,...
54706209,NaN,NaN,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,POINT (114.12785 22.36753)
55123313,NaN,NaN,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,POINT (114.00613 22.46913)
55997466,NaN,NaN,a4158a52-3c61-4f02-abbb-f035faa6faee,POINT (114.16573 22.25782)
56447442,NaN,NaN,eebeebff-812f-4be2-aaa4-94f86c528427,POINT (114.15000 22.29000)


In [23]:
import folium
# https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html
import geopandas as gpd

# 创建地图中心点坐标
center = [22.3193, 114.1694]  # 香港的纬度和经度

# 创建地图对象
m = folium.Map(location=center, zoom_start=12)

# 将建筑物图层添加到地图上
folium.GeoJson(unmapped[:1000]).add_to(m)

# # 保存地图为HTML文件
# m.save("map.html")

m

In [ ]:
device_building_home_df = merged[['device_id','BUILDINGST']]
device_building_home_df

,device_id,BUILDINGST
310,9d84fd00-873a-4473-9c17-42bbf0402455,246850
483,7da1b9a7-3908-475e-a766-638b5d760eb7,186396
573,4b625d9c-fc79-465f-8315-8f39cca15e0e,246925
698,8f47afdb-70aa-438b-a65b-c23e70f1a7be,174391
805,eb9023f1-b06a-4e4f-8a53-cae203f32b24,196917
...,...,...
338948,0efd6c0d-6f95-47b9-a4b6-f0724a970a8f,60539
339357,908023fa-e3a1-4a58-8b4c-c8b358ce252a,343166
339447,9c63161a-3895-4dc9-820d-2de8fbeae333,45990
339848,b2a074a2-fd10-463b-adc3-14288ae32f9c,138660


In [ ]:
## 保存数据：device_building_home
device_building_home_df.to_csv(path + "device_building_home.csv", index=False)